In [78]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator

from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.externals import six
from sklearn.pipeline import _name_estimators
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

from sklearn.feature_selection import RFECV
from sklearn.metrics import log_loss
from sklearn.cross_validation import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
# load data
train = pd.read_csv('C:/Users/Abhishek/Desktop/ml_comps/applied-data-analysis/Applied-Data-Analysis/animal-shelter/data/train.csv/train.csv', parse_dates=['DateTime'])
test = pd.read_csv('C:/Users/Abhishek/Desktop/ml_comps/applied-data-analysis/Applied-Data-Analysis/animal-shelter/data/test.csv/test.csv', parse_dates=['DateTime'])
sub = pd.read_csv('C:/Users/Abhishek/Desktop/ml_comps/applied-data-analysis/Applied-Data-Analysis/animal-shelter/data/sample_submission.csv/sample_submission.csv')

In [119]:
data = pd.concat([train, test])

In [121]:
data['day'] = data.date_time.dt.day
data['month'] = data.date_time.dt.month
data['year'] = data.date_time.dt.year
data['week_day'] = data.date_time.dt.weekday
data['hour'] = data.date_time.dt.hour
data['minute'] = data.date_time.dt.minute

In [123]:
pd.crosstab(data.outcome_type, data.hour)

hour,0,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
outcome_type,,,,,,,,,,,,,,,,,,,,
Adoption,37,1,7,79,143,114,136,530,863,798,832,843,1121,2438,2139,609,56,15,4,4
Died,5,0,1,5,20,25,19,21,21,16,12,11,11,12,14,2,2,0,0,0
Euthanasia,3,0,0,5,62,95,102,125,157,146,177,195,144,163,131,48,1,0,0,1
Return_to_owner,20,0,1,6,9,10,29,662,493,396,405,424,532,822,805,166,3,1,1,1
Transfer,313,1,1,15,68,1034,119,704,979,1112,1374,1209,882,727,595,258,15,2,0,14


### Exploratory Data Analysis

In [113]:
train[]

,animal_id,name,date_time,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,day,month,year,week_day,is_mixed,is_multi_color,animal_type_indicator,outcome_type_encoded
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,3,365.0,Shetland Sheepdog Mix,Brown/White,12,2,2014,2,1,1,1,3
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,4,365.0,Domestic Shorthair Mix,Cream Tabby,13,10,2013,6,1,0,0,2
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,3,2.0,Pit Bull Mix,Blue/White,31,1,2015,5,1,1,1,0
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,2,3.0,Domestic Shorthair Mix,Blue Cream,11,7,2014,4,1,0,0,4
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,3,2.0,Lhasa Apso/Miniature Poodle,Tan,15,11,2013,4,0,0,1,4


In [114]:
test.head()

,id,name,date_time,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,day,month,year,week_day,is_mixed,is_multi_color,animal_type_indicator
0,1,Summer,2015-10-12 12:15:00,Dog,1,300.0,Labrador Retriever Mix,Red/White,12,10,2015,0,1,1,1
1,2,Cheyenne,2014-07-26 17:59:00,Dog,4,2.0,German Shepherd/Siberian Husky,Black/Tan,26,7,2014,5,0,1,1
2,3,Gus,2016-01-13 12:20:00,Cat,3,365.0,Domestic Shorthair Mix,Brown Tabby,13,1,2016,2,1,0,0
3,4,Pongo,2013-12-28 18:12:00,Dog,2,120.0,Collie Smooth Mix,Tricolor,28,12,2013,5,1,0,1
4,5,Skooter,2015-09-24 17:59:00,Dog,3,2.0,Miniature Poodle Mix,White,24,9,2015,3,1,0,1
